<a href="https://colab.research.google.com/github/fernandofsilva/desafio-alelo/blob/main/notebooks/RF_Regressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Random Forest Classifier

## Setup

In [ ]:
# Data handling
import pandas as pd

# Modules for feature engineering and modelling
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import plot_confusion_matrix, r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from math import sqrt

## Load data and merge

In [ ]:
# Characters powers
powers = pd.read_csv("drive/MyDrive/Datasets/super_hero_powers.csv")

# Characters information
info = pd.read_csv(
    "drive/MyDrive/Datasets/heroes_information.csv",
    index_col=0, 
    na_values=['-', -99.0]
)

# Merge databases
data = info.merge(right=powers, how='inner', left_on='name', right_on='hero_names')
data.drop('hero_names', axis=1, inplace=True)

# Removed characters without weight
data = data[data['Weight'].notnull()]

## Feature Engineering

### Class for variable selection

In [ ]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.columns]

### Define columns according to each

In [ ]:
# Categorical columns name
categorical = list(info.select_dtypes('object').columns)[1:]

# Numeric columns name
numerical = list(info.select_dtypes('number').columns)
numerical.remove('Weight')

# Boolean columns name
boolean = list(powers.select_dtypes('bool').columns)

### Construct feature engineering graph

In [ ]:
# Define categorical pipeline
cat_pipe = Pipeline(
    [('selector', ColumnSelector(categorical)),
     ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
     ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))])

# Define numerical pipeline
num_pipe = Pipeline(
    [('selector', ColumnSelector(numerical)),
     ('imputer', SimpleImputer(strategy='median')),
     ('scaler', StandardScaler())])

# Define boolean pipeline
bool_pipe = Pipeline(
    [('selector', ColumnSelector(boolean)),
     ('pca', PCA(n_components=0.95))])

# Fit column transformer to training data
preprocessor = FeatureUnion(
    [('categorical', cat_pipe),
     ('numerical', num_pipe),
     ('boolean', bool_pipe)])

### Transform the target

Unfornately, Sklearn pipeline doen't have support to add a transformation in the target variable like the ReshapeEndpoint in Tensorflow.

In [ ]:
# Format the target variable
y = data['Weight']

## Model

In [ ]:
# Combine categorical and numerical pipeline with the model
model = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(min_samples_leaf=2))
])

### Fit and predict

The function cross_val_predict already has stratify the target variable, in other words, the function balance the target variable between the Kfolds.

In [ ]:
model.fit(data, y)
y_pred = cross_val_predict(model, data, y, cv=10)

## Metrics

### R Square

In [ ]:
print(f'R Square: {r2_score(y, y_pred):.4f}')

R Square: 0.2725


### Root Mean Square Error(RMSE)

In [ ]:
print(f'RMSE: {sqrt(mean_squared_error(y, y_pred)):.2f}')

RMSE: 90.74


### Mean Absolute Error(MAE)

In [ ]:
print(f'MAE: {mean_absolute_error(y, y_pred):.2f}')

MAE: 41.41
